In [1]:
import numpy as np
import pandas as pd
import opensim as osim
from connections import AWS
from biomech.algorithms import diff_three_point
from biomech.opensim.inverse_kinematics import load_mot_file

$\textbf{OpenSim: Joint Reaction Analysis (Development)}$

In [3]:
# update model motion (coords & bodies) for all time points
def update_model_motion(
        model: osim.Model,
        state: osim.State,
        throwing_hand: str,
        q: pd.DataFrame,
        modeled_angles: list[str] = [
            'arm_flex', 
            'arm_add',
            'arm_rot',
            'elbow_flex',
            'pro_sup',
            'wrist_flex',
            'wrist_dev'
    ],
) -> tuple[
    osim.Model, dict[str, dict[str, dict[str, np.ndarray]]]
]:
    # get number of timepoints in trial
    num_samples = len(q)

    # compute q_dot (ignore time column)
    q_dot = diff_three_point(q.iloc[:, 1:])

    # update joint angle cols based on throwing hand
    match throwing_hand:
        case 'left':
            modeled_angles = [col + '_l' for col in modeled_angles]
        case 'right':
            modeled_angles = [col + '_r' for col in modeled_angles]
    
    # iterate over samples
    for k in range(num_samples):
        
        # iterate through model coordinates at each time point
        for _, coord in enumerate(model.getCoordinateSet()):
            # check if coordinate was saved in IK results
            if coord.getName() in modeled_angles:

                # get joint angle, velocity (in degrees)
                q_val = q.loc[k, coord.getName()]
                q_dot_val = q_dot.loc[k, 'diff_' + coord.getName()]

                # save joint angle, velocity (in radians)
                coord.setValue(state, q_val)
                coord.setSpeedValue(state, q_dot_val)

        # update model state
            # moved outside loop
            # also added acceleration, dynamics for CoM acceleration calcs
        model.realizePosition(state)
        model.realizeVelocity(state)

    return model



In [4]:
# load scaled model
model = osim.Model('2609_scaled_model.osim')
model.initSystem()

# update model motion
q = load_mot_file('2609_01_ik.mot')
model_motion = update_model_motion(
    model=model,
    state=model.getWorkingState(),
    throwing_hand='right',
    q=q
)

[info] Loaded model models_arm_right_scaled from file 2609_scaled_model.osim
[warning] Couldn't find file 'humerus_rv.vtp'.
[warning] Couldn't find file 'ulna_rv.vtp'.
[warning] Couldn't find file 'radius_rv.vtp'.
[warning] Couldn't find file 'pisiform_rvs.vtp'.
[warning] Couldn't find file 'lunate_rvs.vtp'.
[warning] Couldn't find file 'scaphoid_rvs.vtp'.
[warning] Couldn't find file 'triquetrum_rvs.vtp'.
[warning] Couldn't find file 'hamate_rvs.vtp'.
[warning] Couldn't find file 'capitate_rvs.vtp'.
[warning] Couldn't find file 'trapezoid_rvs.vtp'.
[warning] Couldn't find file 'trapezium_rvs.vtp'.
[warning] Couldn't find file 'metacarpal2_rvs.vtp'.
[warning] Couldn't find file 'index_proximal_rvs.vtp'.
[warning] Couldn't find file 'index_medial_rvs.vtp'.
[warning] Couldn't find file 'index_distal_rvs.vtp'.
[warning] Couldn't find file 'metacarpal3_rvs.vtp'.
[warning] Couldn't find file 'middle_proximal_rvs.vtp'.
[warning] Couldn't find file 'middle_medial_rvs.vtp'.
[warning] Couldn't 

/Users/cmoore/Library/Mobile Documents/com~apple~CloudDocs/Human RITHM/GitHub/pitch-ml/packages/biomech/opensim/inverse_kinematics.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, skiprows=10)


In [6]:
""" SETUP JOINT REACTION ANALYSIS """
# initialize analysis
jra = osim.JointReaction()
jra.setName("ElbowLoad")

# set joints and forces to report
joint_names = osim.ArrayStr()
joint_names.append("elbow_r")           # NOTE: this depends on throwing hand
jra.setJointNames(joint_names)

# set body for analysis (child: ulna)
    # TODO: check?
on_body = osim.ArrayStr()
on_body.append("child")
jra.setOnBody(on_body)

# set frame for analysis (should be child)
in_frame = osim.ArrayStr()
in_frame.append("child")
jra.setInFrame(in_frame)

# add analysis to model
model_motion.addAnalysis(jra)

In [9]:
# Reinitialize model and state
state = model.initSystem()
model.realizeDynamics(state)

# Load and prep kinematic data
q = load_mot_file("2609_01_ik.mot")
q_dot = diff_three_point(q.iloc[:, 1:])

# Add time column for export
results = []

# Loop through all time steps and evaluate manually
for i, time in enumerate(q['time']):
    state.setTime(time)

    for coord in model.getCoordinateSet():
        coord_name = coord.getName()
        if coord_name in q.columns:
            coord.setValue(state, q.loc[i, coord_name])
            coord.setSpeedValue(state, q_dot.loc[i, 'diff_' + coord_name])

    model.realizeVelocity(state)

    # Manually evaluate JointReactionAnalysis at this state
    jra.step(state, int(i))  # `int(i)` needed as step expects int


/Users/cmoore/Library/Mobile Documents/com~apple~CloudDocs/Human RITHM/GitHub/pitch-ml/packages/biomech/opensim/inverse_kinematics.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, skiprows=10)


In [26]:
""" SETUP ANALYSIS TOOL """
analyze_tool = osim.AnalyzeTool()
analyze_tool.setName("ElbowLoad")
analyze_tool.setModel(model_motion)

# set IK trial path
analyze_tool.setStatesFileName("2609_01_ik.mot")

# set start/end time
mot_data = load_mot_file("2609_01_ik.mot")
analyze_tool.setStartTime(mot_data['time'].values[0])
analyze_tool.setFinalTime(mot_data['time'].values[-1])

# optional: IK filtering
    # NOTE: currently set to 18 to match marker filtering
analyze_tool.setLowpassCutoffFrequency(18)


/Users/cmoore/Library/Mobile Documents/com~apple~CloudDocs/Human RITHM/GitHub/pitch-ml/packages/biomech/opensim/inverse_kinematics.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, skiprows=10)


In [28]:
analyze_tool.run()

[info] No external loads will be applied (external loads file not specified).

Exception:
  analyzeTool.verifyControlsStates: ERROR- a storage object containing the
  time histories of states was not specified.
  file= /Users/ayman/miniconda3/conda-bld/opensim_1686262992860/work/OpenSim/Tools/AnalyzeTool.cpp
  line= 478




RuntimeError: std::exception in 'bool OpenSim::AnalyzeTool::run()': analyzeTool.verifyControlsStates: ERROR- a storage object containing the time histories of states was not specified.